In [41]:
import pandas as pd
import numpy as np

#CARGA DE DATOS
archivo = "../data/raw/aprehendidos_detenidos/dataset/mdi_detenidosaprehendidos_pm_2025_enero_octubre.xlsx"

destino = "../data/interim/aprehendidos_detenidos/"

try:
    print("Cargando archivo Excel... esto puede tardar unos segundos...")
    df_aprehendidos = pd.read_excel(
        archivo,
        sheet_name=1,
        dtype={'codigo_parroquia': str},
        engine='openpyxl'
    )
    print(f"Registros totales: {len(df_aprehendidos)}")
except Exception as e:
    print("Error cargando el Excel:", e)


Cargando archivo Excel... esto puede tardar unos segundos...
Registros totales: 61149


In [42]:
df_aprehendidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61149 entries, 0 to 61148
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   codigo_iccs                  61149 non-null  object        
 1   tipo                         61149 non-null  object        
 2   estado_civil                 61149 non-null  object        
 3   estatus_migratorio           61149 non-null  object        
 4   edad                         61149 non-null  object        
 5   sexo                         61149 non-null  object        
 6   genero                       61149 non-null  object        
 7   nacionalidad                 61149 non-null  object        
 8   autoidentificacion_etnica    61149 non-null  object        
 9   nivel_de_instruccion         61149 non-null  object        
 10  condicion                    61149 non-null  object        
 11  movilizacion                 61149 non-nu

In [43]:
df_aprehendidos.head()


,codigo_iccs,tipo,estado_civil,estatus_migratorio,edad,sexo,genero,nacionalidad,autoidentificacion_etnica,nivel_de_instruccion,...,nombre_subcircuito,codigo_provincia,nombre_provincia,codigo_canton,nombre_canton,codigo_parroquia,nombre_parroquia,presunta_infraccion,latitud,longitud
0,SIN_DATO,DETENIDO,SOLTERO/A,NO_APLICA,26,HOMBRE,MASCULINO,ECUATORIANO,MESTIZO/A,BACHILLERATO,...,PORTOVELO 1,7,EL ORO,711,PORTOVELO,071150,PORTOVELO,BOLETAS,"-3,715507018","-79,618370077"
1,100430.01,APREHENDIDO,SOLTERO/A,SE_DESCONOCE,35,HOMBRE,MASCULINO,ECUATORIANO,MESTIZO/A,EDUCACIÓN GENERAL BÁSICA,...,MISAHUALLI 1,15,NAPO,1501,TENA,150155,PUERTO MISAHUALLI,DELITOS CONTRA LOS RECURSOS MINEROS,"-1,0466604","-77,7429673"
2,SIN_DATO,DETENIDO,SOLTERO/A,REGULAR,45,HOMBRE,MASCULINO,ECUATORIANO,MESTIZO/A,BACHILLERATO,...,FLORESTA 1,9,GUAYAS,901,GUAYAQUIL,090150,GUAYAQUIL-FLORESTA,BOLETAS,"-2,260877405","-79,87844749"
3,060124.01,DETENIDO,CASADO/A,SE_DESCONOCE,39,HOMBRE,MASCULINO,ECUATORIANO,SE_DESCONOCE,NO_APLICA,...,LA CUATRO MIL 1,7,EL ORO,701,MACHALA,070150,MACHALA,DELITOS POR LA PRODUCCIÓN O TRÁFICO ILÍCITO DE...,"-3,270165668","-79,953237234"
4,060124.01,APREHENDIDO,SOLTERO/A,NO_APLICA,66,MUJER,FEMENINO,ECUATORIANO,MESTIZO/A,NO_APLICA,...,VALENCIA 1,12,LOS RÍOS,1211,VALENCIA,121150,VALENCIA,DELITOS POR LA PRODUCCIÓN O TRÁFICO ILÍCITO DE...,"-0,9511309","-79,3588651"


In [44]:
# NORMALIZACIÓN DE DATOS DE LATITUD Y LONGITUD
for col in ["latitud", "longitud"]:
    df_aprehendidos[col] = (
        df_aprehendidos[col]
        .astype(str)
        .str.replace(",", ".", regex=False)
    )
    df_aprehendidos[col] = pd.to_numeric(df_aprehendidos[col], errors="coerce")

# ELIMINAR COLUMNAS INVÁLIDAS
df_aprehendidos = df_aprehendidos.dropna(subset=["latitud", "longitud"])
df_aprehendidos = df_aprehendidos[(df_aprehendidos["latitud"] != 0) & (df_aprehendidos["longitud"] != 0)]
print(f"Registros después de limpiar latitud/longitud inválidas: {len(df_aprehendidos)}")

Registros después de limpiar latitud/longitud inválidas: 61149


In [45]:
# PROCESAR FECHA Y HORA
try:
    df_aprehendidos["fecha"] = pd.to_datetime(df_aprehendidos["fecha_detencion_aprehension"], errors="coerce")

    #HORA COMO STRING LIMPIO
    df_aprehendidos["hora"] = (
        df_aprehendidos["hora_detencion_aprehension"]
        .astype(str)
        .str.replace(" ", "")
        .str.strip()
    )


except Exception as e:
    print("Advertencia procesando fecha/hora:", e)

# ELIMINAR COLUMNAS ORIGINALES DE FECHA Y HORA    
df_aprehendidos.drop(columns=["fecha_detencion_aprehension", "hora_detencion_aprehension"], inplace=True)   
print(f"Registros después de limpiar latitud/longitud inválidas: {len(df_aprehendidos)}")



Registros después de limpiar latitud/longitud inválidas: 61149


In [46]:
df_aprehendidos.head()

,codigo_iccs,tipo,estado_civil,estatus_migratorio,edad,sexo,genero,nacionalidad,autoidentificacion_etnica,nivel_de_instruccion,...,nombre_provincia,codigo_canton,nombre_canton,codigo_parroquia,nombre_parroquia,presunta_infraccion,latitud,longitud,fecha,hora
0,SIN_DATO,DETENIDO,SOLTERO/A,NO_APLICA,26,HOMBRE,MASCULINO,ECUATORIANO,MESTIZO/A,BACHILLERATO,...,EL ORO,711,PORTOVELO,071150,PORTOVELO,BOLETAS,-3.715507,-79.618370,2025-02-03,08:00:00
1,100430.01,APREHENDIDO,SOLTERO/A,SE_DESCONOCE,35,HOMBRE,MASCULINO,ECUATORIANO,MESTIZO/A,EDUCACIÓN GENERAL BÁSICA,...,NAPO,1501,TENA,150155,PUERTO MISAHUALLI,DELITOS CONTRA LOS RECURSOS MINEROS,-1.046660,-77.742967,2025-05-16,13:40:00
2,SIN_DATO,DETENIDO,SOLTERO/A,REGULAR,45,HOMBRE,MASCULINO,ECUATORIANO,MESTIZO/A,BACHILLERATO,...,GUAYAS,901,GUAYAQUIL,090150,GUAYAQUIL-FLORESTA,BOLETAS,-2.260877,-79.878447,2025-05-20,13:30:00
3,060124.01,DETENIDO,CASADO/A,SE_DESCONOCE,39,HOMBRE,MASCULINO,ECUATORIANO,SE_DESCONOCE,NO_APLICA,...,EL ORO,701,MACHALA,070150,MACHALA,DELITOS POR LA PRODUCCIÓN O TRÁFICO ILÍCITO DE...,-3.270166,-79.953237,2025-04-11,17:00:00
4,060124.01,APREHENDIDO,SOLTERO/A,NO_APLICA,66,MUJER,FEMENINO,ECUATORIANO,MESTIZO/A,NO_APLICA,...,LOS RÍOS,1211,VALENCIA,121150,VALENCIA,DELITOS POR LA PRODUCCIÓN O TRÁFICO ILÍCITO DE...,-0.951131,-79.358865,2025-09-18,15:30:00


In [47]:
df_aprehendidos.to_csv(destino + "aprehendidos_detenidos_limpio.csv", index=False)